# RDD operacje

In [1]:
# import modułu pyspark
import pyspark

# powolujemy kontekst sparka do życia
# local[*] - jako instancja lokalna, na wątkach tylu ile jest dostępnych
sc = pyspark.SparkContext('local[*]')

In [2]:
# tablica dwóch napisów
data = ["mike, tom, lukas", "pati, sandra, tom"]
print(data)

['mike, tom, lukas', 'pati, sandra, tom']


In [3]:
# najprostszy sposób aby uzyskać rozproszony typ danych w Sparku:
distributedData = sc.parallelize(data)

# RDD - Resilient Distributed Datasets

In [4]:
type(distributedData)

pyspark.rdd.RDD

In [5]:
distributedData.collect()

['mike, tom, lukas', 'pati, sandra, tom']

### Funkcje w Sparku

Przetwarzanie w Sparku silnie wykorzystuje wykorzystywanie funkcji lambda. lambda - są to anonimowe funkcje, które używa się jako argumentu metod Sparka. Oczywiście nie ma przymusu używania labda. Można w ich miejsce używać standardowych funkcji.

map(funkcja) - zwraca nowy RDD zawierający elementy, które zostały zmapowane funkcją funkcja. 
Przykład przy użyciu `lambda`.

    metoda: `collect()` zbiera wszystkie rozproszone wyniki i wyświetla na maszynie lokalnej.

In [6]:
distributedData.map(lambda line: line.split(", ")).collect()

[['mike', 'tom', 'lukas'], ['pati', 'sandra', 'tom']]

Ten sam kod, ale przy użyciu standardowej funkcji:

In [7]:
def splitText(text):
    return text.split(", ")

mapped = distributedData.map(splitText)
mapped.collect()

[['mike', 'tom', 'lukas'], ['pati', 'sandra', 'tom']]

Funkcji manipulujących wynikami jest więcej przykłądowo:

In [8]:
# Wyświetl 3 pierwsze elementy:
mapped.take(3)

[['mike', 'tom', 'lukas'], ['pati', 'sandra', 'tom']]

In [9]:
# pierwszy element:
mapped.first()

['mike', 'tom', 'lukas']

In [10]:
# 2 najwięsze elementy
mapped.top(2)

[['pati', 'sandra', 'tom'], ['mike', 'tom', 'lukas']]

In [11]:
# policz elementy
mapped.count()

2

In [12]:
# pobierz losowo 1 element ze zbioru
mapped.takeSample(False, 1)

[['pati', 'sandra', 'tom']]

In [13]:
# wybierz elementy, gdzie prawdopodobieństwo wybrania danego elementu to 0.5:
mapped.sample(False, 0.5).collect()

[['mike', 'tom', 'lukas'], ['pati', 'sandra', 'tom']]

Często mamy taką sytuację, że same elementy w RDD są kolekcjami elementów. Tak jest w tym wypadku. Często należy wtedy rozpakować kolekcje do osobnych elementów. Robi to funkcja `flatMap(funkcja)` (tzw. spłaszanie kolekcji)
`flatMap(func)` - podobne do map, ale dodatkowo każdy element kolekcji zostaje rozpakowany (1 element wejściowy zmapowany zostaje na wiele elementów wyjściowych). Widzisz różnicę?

In [14]:
rdd1 = distributedData.map(lambda line: line.split(", "))
rdd1.collect()

[['mike', 'tom', 'lukas'], ['pati', 'sandra', 'tom']]

In [15]:
rdd2 = rdd1.flatMap(lambda word: word)
rdd2.collect()

['mike', 'tom', 'lukas', 'pati', 'sandra', 'tom']

In [16]:
rdd3 = rdd2.map(lambda word: (word, len(word)))
rdd3.collect()

[('mike', 4), ('tom', 3), ('lukas', 5), ('pati', 4), ('sandra', 6), ('tom', 3)]

Porównaj z wynikiem zwykłego map:

In [17]:
mapped.collect()

[['mike', 'tom', 'lukas'], ['pati', 'sandra', 'tom']]

Oczywiście opearcje `map` mogą wykonywać bardziej złożone operacje:

In [18]:
distributedData.map(lambda data: (len(data), data)).collect()

[(16, 'mike, tom, lukas'), (17, 'pati, sandra, tom')]

Często zachodzi sytuacja w której chcemy odrzucić pewne elementy podczas przetwarzania elementów naszego zbioru danych. Musimy więc je odfiltrować i zostawić tylko te, które spełniają pewne kryteria. Robi to metoda `filter(funkcja)`:


In [19]:
distributedData.collect()

['mike, tom, lukas', 'pati, sandra, tom']

In [20]:
# zostaw tylko te elementy zawierające imię sandra
rdd4 = distributedData.filter(lambda line: "sandra" in line)

In [21]:
rdd4.take(2)

['pati, sandra, tom']

In [22]:
liczby = sc.parallelize([1,2,3,4,4]) # liczba 4 występuje kilkukrotnie

In [23]:
# wartości unikalne? Proszę bardzo!
distinct = liczby.distinct().collect()
distinct

[1, 2, 3, 4]

W przetwarzaniu Big Data bardzo ważnym elementem obliczeń jest realizowanie operacji [MapReduce](https://pl.wikipedia.org/wiki/MapReduce). Operacja ta polega na wykonaniu mapowania a następnie na jej wyniku wykonaniu operacji redukcji. 

Operacja redukcji polaga na przekazaniu funkcji, której wynikiem będzie złączenie dowolnych dwóch elementów danego zbioru. Wyobraźmy sobie taką tablicę: [1,2,3,4]. Można zaprojektować funkcję redukcji której wynikiem będzie suma elementów danej tablicy:

In [24]:
rdd1 = sc.parallelize(range(1, 5))
rdd1.collect()

[1, 2, 3, 4]

In [25]:
rdd2 = rdd1.map(lambda x: x*2)

In [26]:
rdd2.reduce(lambda x,y: x+y)

20

In [27]:
# albo ładniej:
rdd1 \
    .map(lambda x: x*2) \
    .reduce(lambda x,y: x+y)

20

Pamiętaj proszę, że niektóre metody wykonywane na RDD zwracają kolejny RDD (np. map, filter...) zaś inne od razu wynik (np. collect, reduce, ...). Omówimy te dwa typy nieco później.

`Redukcja`, albo to samo za pomocą `fold`

In [28]:
rdd1.reduce(lambda x, y: x + y)

10

In [29]:
rdd1.fold(0, lambda x, y: x + y)

10

Robiąc obliczenia na klastrze bardzo ważną sprawą jest wykonywanie `lokalności danych`. Ważnym jest aby najpierw wykonać obliczenia na danych zgromadzonych na danej maszynie a dopiero jako jeden wynik z maszyny zagregować po wielu maszynach. Taką funkcją jest agregate który umożliwia wykonanie 2 etapów.

`aggregate` - agreguje elementy najpierw z użyciem accumulatora, a potem gdy wyniki spływają to combinera. Parametry metody: `zero element, accumulator, combinerAccumulator`

In [30]:
rdd1.collect()

[1, 2, 3, 4]

In [31]:
rdd1.aggregate(
    (0,0),
    lambda acc, value: (acc[0] + value, acc[1] + 1),
    lambda acc1, acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1])
)


(10, 4)

In [32]:
print(rdd1.count())
print(rdd1.countByValue())

4
defaultdict(<class 'int'>, {1: 1, 2: 1, 3: 1, 4: 1})


In [33]:
def f(x): 
    print(x)

print('Print ver1:')
rdd2.foreach(f)

print('Print ver2:')
for k in rdd2.collect():
    f(k)
    
# dlaczego `Print ver1` puste?

Print ver1:
Print ver2:
2
4
6
8


first() - zwraca pierwszy element RDD
take(n) - zwraca n elementów RDD
takeSample(withReplacement, n) - zwraca próbkę danych o rozmiarze n, z lub bez powtórzeń
takeOrdered(n) - zwraca posortowane n elementów


Użyjmy pliku: `cars.csv`

"company","name","mpg","cyl","disp","hp","drat","wt","qsec","vs","am","gear","carb"

Niektóre opisy danych:
* mpg - miles per galon
* cyl - cylynders
* hp - horse power
* gear - gears
* carb - carburetor

In [34]:
filedata = sc.textFile("cars.csv")
filedata.count()

32

In [35]:
filedata.first()

'Mazda,"Mazda RX4",21,6,160,110,3.9,2.62,16.46,0,1,4,4'

In [36]:
rows = filedata.map(lambda line: line.split(","))
rows.first()

['Mazda',
 '"Mazda RX4"',
 '21',
 '6',
 '160',
 '110',
 '3.9',
 '2.62',
 '16.46',
 '0',
 '1',
 '4',
 '4']

`groupByKey`
wywołany na RDD zawierający (K, V) pary, zwraca zbiór (K, Iterable<V>) par.

In [37]:
rows.take(2)

[['Mazda',
  '"Mazda RX4"',
  '21',
  '6',
  '160',
  '110',
  '3.9',
  '2.62',
  '16.46',
  '0',
  '1',
  '4',
  '4'],
 ['Mazda',
  '"Mazda RX4 Wag"',
  '21',
  '6',
  '160',
  '110',
  '3.9',
  '2.875',
  '17.02',
  '0',
  '1',
  '4',
  '4']]

In [38]:
spalanie = rows.map(lambda data: (data[0], data[1]))
spalanie.collect()

[('Mazda', '"Mazda RX4"'),
 ('Mazda', '"Mazda RX4 Wag"'),
 ('Datsun', '"Datsun 710"'),
 ('Hornet', '"Hornet 4 Drive"'),
 ('Hornet', '"Hornet Sportabout"'),
 ('Plymouth', '"Valiant"'),
 ('Plymouth', '"Duster 360"'),
 ('Merc', '"Merc 240D"'),
 ('Merc', '"Merc 230"'),
 ('Merc', '"Merc 280"'),
 ('Merc', '"Merc 280C"'),
 ('Merc', '"Merc 450SE"'),
 ('Merc', '"Merc 450SL"'),
 ('Merc', '"Merc 450SLC"'),
 ('Cadillac', '"Cadillac Fleetwood"'),
 ('Lincoln', '"Lincoln Continental"'),
 ('Chrysler', '"Chrysler Imperial"'),
 ('Fiat', '"Fiat 128"'),
 ('Honda', '"Honda Civic"'),
 ('Toyota', '"Toyota Corolla"'),
 ('Toyota', '"Toyota Corona"'),
 ('Dodge', '"Dodge Challenger"'),
 ('AMC', '"AMC Javelin"'),
 ('Chevrolet', '"Camaro Z28"'),
 ('Pontiac', '"Pontiac Firebird"'),
 ('Fiat', '"Fiat X1-9"'),
 ('Porsche', '"Porsche 914-2"'),
 ('Lotus', '"Lotus Europa"'),
 ('Ford', '"Ford Pantera L"'),
 ('Ferrari', '"Ferrari Dino"'),
 ('Maserati', '"Maserati Bora"'),
 ('Volvo', '"Volvo 142E"')]

In [39]:
partialResult = spalanie.groupByKey()
partialResult.collect()

[('Hornet', <pyspark.resultiterable.ResultIterable at 0x7faa4c84e550>),
 ('Plymouth', <pyspark.resultiterable.ResultIterable at 0x7faa4c84e588>),
 ('Lincoln', <pyspark.resultiterable.ResultIterable at 0x7faa4c84e668>),
 ('AMC', <pyspark.resultiterable.ResultIterable at 0x7faa4c84e5c0>),
 ('Pontiac', <pyspark.resultiterable.ResultIterable at 0x7faa4c84e748>),
 ('Porsche', <pyspark.resultiterable.ResultIterable at 0x7faa4c84e7b8>),
 ('Ford', <pyspark.resultiterable.ResultIterable at 0x7faa4c84e828>),
 ('Ferrari', <pyspark.resultiterable.ResultIterable at 0x7faa4c84e6a0>),
 ('Volvo', <pyspark.resultiterable.ResultIterable at 0x7faa4c84e908>),
 ('Mazda', <pyspark.resultiterable.ResultIterable at 0x7faa4c84e6d8>),
 ('Datsun', <pyspark.resultiterable.ResultIterable at 0x7faa4c84e898>),
 ('Merc', <pyspark.resultiterable.ResultIterable at 0x7faa4c84ea58>),
 ('Cadillac', <pyspark.resultiterable.ResultIterable at 0x7faa4c84e9b0>),
 ('Chrysler', <pyspark.resultiterable.ResultIterable at 0x7faa4c8

In [40]:
partialResult.map(lambda data_tuple : (data_tuple[0], list(data_tuple[1]))).collect()

[('Hornet', ['"Hornet 4 Drive"', '"Hornet Sportabout"']),
 ('Plymouth', ['"Valiant"', '"Duster 360"']),
 ('Lincoln', ['"Lincoln Continental"']),
 ('AMC', ['"AMC Javelin"']),
 ('Pontiac', ['"Pontiac Firebird"']),
 ('Porsche', ['"Porsche 914-2"']),
 ('Ford', ['"Ford Pantera L"']),
 ('Ferrari', ['"Ferrari Dino"']),
 ('Volvo', ['"Volvo 142E"']),
 ('Mazda', ['"Mazda RX4"', '"Mazda RX4 Wag"']),
 ('Datsun', ['"Datsun 710"']),
 ('Merc',
  ['"Merc 240D"',
   '"Merc 230"',
   '"Merc 280"',
   '"Merc 280C"',
   '"Merc 450SE"',
   '"Merc 450SL"',
   '"Merc 450SLC"']),
 ('Cadillac', ['"Cadillac Fleetwood"']),
 ('Chrysler', ['"Chrysler Imperial"']),
 ('Fiat', ['"Fiat 128"', '"Fiat X1-9"']),
 ('Honda', ['"Honda Civic"']),
 ('Toyota', ['"Toyota Corolla"', '"Toyota Corona"']),
 ('Dodge', ['"Dodge Challenger"']),
 ('Chevrolet', ['"Camaro Z28"']),
 ('Lotus', ['"Lotus Europa"']),
 ('Maserati', ['"Maserati Bora"'])]

`reduceByKey(func)`

działa na (K,V) parach, funkcja redukująca musi być typu (V,V) -> V

In [41]:
companyHp = rows.map(lambda car: (car[0], int(car[5])))
companyHp.reduceByKey(lambda x,y: x + y).collect()

[('Hornet', 285),
 ('Plymouth', 350),
 ('Lincoln', 215),
 ('AMC', 150),
 ('Pontiac', 175),
 ('Porsche', 91),
 ('Ford', 264),
 ('Ferrari', 175),
 ('Volvo', 109),
 ('Mazda', 220),
 ('Datsun', 93),
 ('Merc', 943),
 ('Cadillac', 205),
 ('Chrysler', 230),
 ('Fiat', 132),
 ('Honda', 52),
 ('Toyota', 162),
 ('Dodge', 150),
 ('Chevrolet', 245),
 ('Lotus', 113),
 ('Maserati', 335)]

sortByKey

wywołany na zbiorze (K,V) par gdzie K implementuje `Ordered` - zwraca zbiór RDD (K, V) par posortowanych po kluczach w rosnącym (lub malejącym) porządku. 

In [42]:
hpCompany = rows.map(lambda car: (int(car[5]), car[1]))
hpCompany.sortByKey().collect()

# sortByKey() == sortByKey(ascending=True)

[(52, '"Honda Civic"'),
 (62, '"Merc 240D"'),
 (65, '"Toyota Corolla"'),
 (66, '"Fiat 128"'),
 (66, '"Fiat X1-9"'),
 (91, '"Porsche 914-2"'),
 (93, '"Datsun 710"'),
 (95, '"Merc 230"'),
 (97, '"Toyota Corona"'),
 (105, '"Valiant"'),
 (109, '"Volvo 142E"'),
 (110, '"Mazda RX4"'),
 (110, '"Mazda RX4 Wag"'),
 (110, '"Hornet 4 Drive"'),
 (113, '"Lotus Europa"'),
 (123, '"Merc 280"'),
 (123, '"Merc 280C"'),
 (150, '"Dodge Challenger"'),
 (150, '"AMC Javelin"'),
 (175, '"Hornet Sportabout"'),
 (175, '"Pontiac Firebird"'),
 (175, '"Ferrari Dino"'),
 (180, '"Merc 450SE"'),
 (180, '"Merc 450SL"'),
 (180, '"Merc 450SLC"'),
 (205, '"Cadillac Fleetwood"'),
 (215, '"Lincoln Continental"'),
 (230, '"Chrysler Imperial"'),
 (245, '"Duster 360"'),
 (245, '"Camaro Z28"'),
 (264, '"Ford Pantera L"'),
 (335, '"Maserati Bora"')]

In [43]:
# posortować odwrotnie? Proszę bardzo!
hpCompany.sortByKey(ascending=False).collect()

[(335, '"Maserati Bora"'),
 (264, '"Ford Pantera L"'),
 (245, '"Duster 360"'),
 (245, '"Camaro Z28"'),
 (230, '"Chrysler Imperial"'),
 (215, '"Lincoln Continental"'),
 (205, '"Cadillac Fleetwood"'),
 (180, '"Merc 450SE"'),
 (180, '"Merc 450SL"'),
 (180, '"Merc 450SLC"'),
 (175, '"Hornet Sportabout"'),
 (175, '"Pontiac Firebird"'),
 (175, '"Ferrari Dino"'),
 (150, '"Dodge Challenger"'),
 (150, '"AMC Javelin"'),
 (123, '"Merc 280"'),
 (123, '"Merc 280C"'),
 (113, '"Lotus Europa"'),
 (110, '"Mazda RX4"'),
 (110, '"Mazda RX4 Wag"'),
 (110, '"Hornet 4 Drive"'),
 (109, '"Volvo 142E"'),
 (105, '"Valiant"'),
 (97, '"Toyota Corona"'),
 (95, '"Merc 230"'),
 (93, '"Datsun 710"'),
 (91, '"Porsche 914-2"'),
 (66, '"Fiat 128"'),
 (66, '"Fiat X1-9"'),
 (65, '"Toyota Corolla"'),
 (62, '"Merc 240D"'),
 (52, '"Honda Civic"')]

In [44]:
# albo sorotowanie w sposób ogólny wg. danych, które sami wskażemy
# w tym przypadku alfabetycznie:
hpCompany.sortBy(ascending=False, keyfunc = lambda data: data[1]).collect()

[(109, '"Volvo 142E"'),
 (105, '"Valiant"'),
 (97, '"Toyota Corona"'),
 (65, '"Toyota Corolla"'),
 (91, '"Porsche 914-2"'),
 (175, '"Pontiac Firebird"'),
 (180, '"Merc 450SLC"'),
 (180, '"Merc 450SL"'),
 (180, '"Merc 450SE"'),
 (123, '"Merc 280C"'),
 (123, '"Merc 280"'),
 (62, '"Merc 240D"'),
 (95, '"Merc 230"'),
 (110, '"Mazda RX4"'),
 (110, '"Mazda RX4 Wag"'),
 (335, '"Maserati Bora"'),
 (113, '"Lotus Europa"'),
 (215, '"Lincoln Continental"'),
 (175, '"Hornet Sportabout"'),
 (110, '"Hornet 4 Drive"'),
 (52, '"Honda Civic"'),
 (264, '"Ford Pantera L"'),
 (66, '"Fiat X1-9"'),
 (66, '"Fiat 128"'),
 (175, '"Ferrari Dino"'),
 (245, '"Duster 360"'),
 (150, '"Dodge Challenger"'),
 (93, '"Datsun 710"'),
 (230, '"Chrysler Imperial"'),
 (245, '"Camaro Z28"'),
 (205, '"Cadillac Fleetwood"'),
 (150, '"AMC Javelin"')]